In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import CSVLoader
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from pydantic import BaseModel, Field
from langchain.chains import LLMChain, create_tagging_chain_pydantic
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import Tool
from langchain.agents import create_tool_calling_agent,AgentExecutor

In [ ]:
llm=ChatOpenAI(model='gpt-3.5-turbo-0613')

In [ ]:
# loader = CSVLoader('./dataset/menu.csv')
# docs = loader.load()

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
# splits = text_splitter.split_documents(docs)

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(documents=splits, embedding=embeddings)

In [ ]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()

In [ ]:
def ask_user(ask_for=['service']):
    first_prompt = ChatPromptTemplate.from_template(
        '''You are a smart waiter at a restaurant. First  you should ask the user in conversational ways whether the user's order is delivery or dine-in.
        Don not ask addtional information just ask whether the order is delivery or dine-in.
        # Example:"Would you like to have a dine-in or delivery?"
        '''
    )          
    
    chain = LLMChain(llm=llm, prompt=first_prompt, verbose=True,memory=memory)
    conv = chain.run(ask_for=ask_for) 
    return conv



In [ ]:
print(ask_user())

In [ ]:
from langchain.chains import ConversationChain

In [ ]:
def create_order(user_query):
    first_prompt = ChatPromptTemplate.from_template(
       """You are a waiter at a restaurant. Below is are some things to ask the user for in a coversation way. 
        you should only ask one question at a time even if you don't get all the info 
        don't ask as a list! First greet the user and welcome the user to our restaurant in a friendly way!
        Explain you need take the food order from the user.
        ### ask_for : ```food_name```
        Current conversation:
        {history}
        Human: {input}
        AI Assistant:
        # """
    )
    
    # info_chain = LLMChain(llm = llm, prompt=first_prompt, verbose=True)
    conversation_chain = ConversationChain(llm= llm, verbose = True,prompt = first_prompt, memory = ConversationBufferMemory(ai_prefix= "AI Assistant"))
    ai_chat = conversation_chain.run(user_query)
    return ai_chat

In [ ]:

def ask_services(user_input):
    service_type=input(ask_user())

    
    
    first_prompt1 = ChatPromptTemplate.from_template(
       f""" You are a smart assistant waiter in a ABC restaurant.After the order, the user is asked whether he/she wants a dine-in or wants a delivery.Your role is mentioned below:
       
        If the user wants delivery then tell them to provide their delivery address(only in case of delivery) 
       
        If the user wants dine-in then tell them to have seat and enjoy the meal. 
        In case if the user wants delivery then tell them to provide their address.
        
       Provide staright forward answer related to whether the order is to be deliver or dine-in and do not ask any unwanted additional information
       
       # ask_for :{service_type}
      """     
      
      
    )


    #  chain = LLMChain(llm=llm, prompt=first_prompt, verbose=True,memory=memory)
    # conv = chain.run(ask_for=ask_for)  # Assuming a default input for now
    # return conv

    chain = LLMChain(llm = llm, prompt=first_prompt1, verbose=True,memory=memory)

    convs = chain.run(ask_for=service_type)


    return convs

In [ ]:
print(ask_services())

In [ ]:
class address(BaseModel):
    address:str=Field(description="This is the address of the customers.")
                                                                                              
    #dine_in:str=Field(description="It is the response of the customer if they did not go with the delivery.")


address_chain=create_tagging_chain_pydantic(address,llm)

In [ ]:
def record_address():
    user_input=input(ask_services())
    chain=create_tagging_chain_pydantic(address,llm)
    response=chain.run(user_input)
    return response

In [ ]:

def ask_services():
    service_type=input(ask_user())

    
    
    first_prompt1 = ChatPromptTemplate.from_template(
       f""" You are a smart assistant waiter in a ABC restaurant.After the order, the user is asked whether he/she wants a dine-in or wants a delivery.Your role is mentioned below:
       
        If the user wants delivery then tell them to provide their delivery address(only in case of delivery) 
       
        If the user wants dine-in then tell them to have seat and enjoy the meal. 
        In case if the user wants delivery then tell them to provide their address.
        
       Provide staright forward answer related to whether the order is to be deliver or dine-in and do not ask any unwanted additional information
       
       # ask_for :{service_type}
      """     
      
      
    )


    #  chain = LLMChain(llm=llm, prompt=first_prompt, verbose=True,memory=memory)
    # conv = chain.run(ask_for=ask_for)  # Assuming a default input for now
    # return conv

    chain = LLMChain(llm = llm, prompt=first_prompt1, verbose=True,memory=memory)

    convs = chain.run(ask_for=service_type)


    return convs

address_tool=Tool(
    name='address',
    func=ask_services,
    description="A tool that is used to record the address of a customers that is provided by the customer."
    
)



# 2. Prompt for agent
address_prompt = ChatPromptTemplate.from_messages(
    [
    (
        "system", 
        "You are a smart assistant that record address. Use the tool for getting the address of the customers."
    ),
    ("human","{input}"),
    ("placeholder","{agent_scratchpad}"),
    ("placeholder","{tool_names}"),
    ("placeholder","{tools}"),
    ]
)



In [ ]:
tools = [address_tool]
agent = create_tool_calling_agent(llm, tools, address_prompt)
agent_executor = AgentExecutor(tools = tools, agent= agent,return_intermediate_steps=False,verbose=True)
agent_executor.invoke("hi")

In [ ]:


receipt_tool = Tool(
        name = "ReceiptCalculator",
        func= calculate_total_price,
        description = "A tool that returns total price when the user asks for receipt or bill."
    )
# memory = ConversationBufferWindowMemory(memory_key = "chat_history", return_messages= True)
prompt_receipt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a helpful assistant. Make sure to use the tool for generating receipt",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
## Construct the Tools agent
tools = [receipt_tool]
agent = create_tool_calling_agent(llm, tools, prompt_receipt)
agent_executor = AgentExecutor(tools = tools, agent= agent)
agent_executor.invoke({"input":"I need my receipt"})

In [ ]:
record_address()

In [ ]:
class dine_in(BaseModel):
    dine_in:str=Field(description="It is the service where user wants to have a dine-in")


